In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType
from pyspark import SparkConf, SparkContext

## Criando a [Spark Session](http://127.0.0.1:4040/)
Configurações padrão utilizadas no _spark-defaults.conf_ em $SPARK_HOME/conf/ 

In [5]:
if 'spark' in locals() or 'spark' in globals():
    spark.stop()

spark = SparkSession \
    .builder \
    .appName("Teste Metastore") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
spark

In [7]:
spark.sql('SHOW DATABASES').show()

+-------------+
|    namespace|
+-------------+
|      default|
|        delta|
|dundermifflin|
+-------------+



In [8]:
path = "s3a://my-bucket/input/sample-data.csv"

schema = StructType() \
        .add("Row ID", IntegerType(), True) \
        .add("Order ID", StringType(), True) \
        .add("Order Date", StringType(), True) \
        .add("Ship Date", StringType(), True) \
        .add("Ship Mode", StringType(), True) \
        .add("Customer ID", StringType(), True) \
        .add("Customer Name", StringType(), True) \
        .add("Segment", StringType(), True) \
        .add("Country", StringType(), True) \
        .add("City", StringType(), True) \
        .add("State", StringType(), True) \
        .add("Postal Code", StringType(), True) \
        .add("Region", StringType(), True) \
        .add("Product ID", StringType(), True) \
        .add("Category", StringType(), True) \
        .add("Sub-Category", StringType(), True) \
        .add("Product Name", StringType(), True) \
        .add("Sales", DoubleType(), True) \
        .add("Quantity", IntegerType(), True) \
        .add("Discount", DoubleType(), True) \
        .add("Profit", DoubleType(), True)

df = spark.read.csv(path = path, schema = schema, header = True)
df.show()

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 0) / 1]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df.write.option("compression", "snappy")\
    .mode('overwrite')\
    .partitionBy('Region')\
    .format('parquet')\
    .saveAsTable('tb_sales_test')

In [ ]:
spark.sql("SHOW TABLES").show()

In [ ]:
spark.catalog.tableExists(tableName = "tb_sales_test")

In [ ]:
spark.sql("SHOW PARTITIONS default.tb_sales_test").show()

In [ ]:
spark.sql("SELECT * FROM default.tb_sales_test LIMIT 10").show()

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS delta")
spark.sql("SHOW DATABASES").show()

In [ ]:
new_cols = [x.replace(' ', '_') for x in df.columns]

df = df.toDF(*new_cols)

In [ ]:
df.printSchema()

In [ ]:
df.write.format("delta").mode("overwrite").partitionBy("Region").saveAsTable("delta.superstore_table")

In [ ]:
spark.catalog.listTables(dbName = 'delta')

In [ ]:
spark.read.table("delta.superstore_table").show(5)

In [ ]:
spark.stop()